# Import All Libaries

In [1]:
# Imports
from tira.third_party_integrations import ensure_pyterrier_is_loaded, persist_and_normalize_run
from tira.rest_api_client import Client
ensure_pyterrier_is_loaded()
import pyterrier as pt

from jnius import autoclass

# Create a REST client to the TIRA platform for retrieving the pre-indexed data.
tira = Client()


import nltk
from nltk.stem import LancasterStemmer
from nltk.stem import WordNetLemmatizer

nltk.download('punkt')
nltk.download('wordnet')

PyTerrier 0.10.0 has loaded Terrier 5.8 (built by craigm on 2023-11-01 18:05) and terrier-helper 0.0.8

No etc/terrier.properties, using terrier.default.properties for bootstrap configuration.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

# Load the Dataset and the Index

In [2]:
# The dataset: the union of the IR Anthology and the ACL Anthology
# This line creates an IRDSDataset object and registers it under the name provided as an argument.
dataset = 'ir-acl-anthology-20240504-training'
pt_dataset = pt.get_dataset(f'irds:ir-lab-sose-2024/{dataset}')


# A (pre-built) PyTerrier index loaded from TIRA
index = tira.pt.index('ir-lab-sose-2024/tira-ir-starter/Index (tira-ir-starter-pyterrier)', pt_dataset)
bm25 = pt.BatchRetrieve(index, wmodel="BM25")

### Stemmer and Lemmatizer  (funktioniert leider nicht)

Variante 1

In [3]:
def stem(t):
    lemmatizer = WordNetLemmatizer()
    return lemmatizer.lemmatize(t)

lemmatizer = WordNetLemmatizer()


indexer = pt.IterDictIndexer("/tmp/index", overwrite=True, stemmer=lemmatizer)
index_ref = indexer.index(pt_dataset.get_corpus_iter())
index = pt.IndexFactory.of(index_ref)

bm25_stem1 = pt.BatchRetrieve(index, wmodel="BM25")



def stem(t):
    stemmer = LancasterStemmer()
    return stemmer.stem(t)

stemmer = LancasterStemmer()

indexer = pt.IterDictIndexer("/tmp/index", overwrite=True, stemmer=stemmer)
index_ref = indexer.index(pt_dataset.get_corpus_iter())
index = pt.IndexFactory.of(index_ref)


bm25_lem = pt.BatchRetrieve(index, wmodel="BM25")

ir-lab-sose-2024/ir-acl-anthology-20240504-training documents:  71%|███████   | 89789/126958 [00:18<00:05, 7378.75it/s]

14:40:22.689 [ForkJoinPool-1-worker-3] WARN org.terrier.structures.indexing.Indexer - Adding an empty document to the index (2020.mir_conference-2020.1) - further warnings are suppressed


ir-lab-sose-2024/ir-acl-anthology-20240504-training documents: 100%|██████████| 126958/126958 [00:22<00:00, 5598.99it/s] 


14:40:30.750 [ForkJoinPool-1-worker-3] WARN org.terrier.structures.indexing.Indexer - Indexed 3 empty documents


ir-lab-sose-2024/ir-acl-anthology-20240504-training documents:  70%|███████   | 89466/126958 [00:13<00:04, 7587.93it/s]

14:40:47.020 [ForkJoinPool-2-worker-3] WARN org.terrier.structures.indexing.Indexer - Adding an empty document to the index (2020.mir_conference-2020.1) - further warnings are suppressed


ir-lab-sose-2024/ir-acl-anthology-20240504-training documents: 100%|██████████| 126958/126958 [00:18<00:00, 6954.63it/s] 


14:40:54.414 [ForkJoinPool-2-worker-3] WARN org.terrier.structures.indexing.Indexer - Indexed 3 empty documents


Variante 2

In [4]:
def stem_text(text):
    stemmer = LancasterStemmer()
    return " ".join([stemmer.stem(word) for word in nltk.word_tokenize(text)])

def lemmatize_text(text):
    lemmatizer = WordNetLemmatizer()
    return " ".join([lemmatizer.lemmatize(word) for word in nltk.word_tokenize(text)])

def document_iterator(docs, transform_func):
    for doc in docs:
        doc["text"] = transform_func(doc["text"])
        yield doc


transform_func = stem_text  # oder lemmatize_text


indexer = pt.IterDictIndexer("/tmp/index", overwrite=True)
index_ref = indexer.index(document_iterator(pt_dataset.get_corpus_iter(), transform_func))

class NLTKStemmerTransformer(pt.Transformer):
    def __init__(self, stemmer):
        self.stemmer = stemmer

    def transform(self, queries):
        queries['query'] = queries['query'].apply(stem_text)
        return queries


stemmer = LancasterStemmer()
nltk_stemmer_transformer = NLTKStemmerTransformer(stemmer)


index = pt.IndexFactory.of(index_ref)


bm25_stem2 = nltk_stemmer_transformer >> pt.BatchRetrieve(index, wmodel="BM25")

ir-lab-sose-2024/ir-acl-anthology-20240504-training documents:  71%|███████   | 90022/126958 [03:00<00:15, 2339.11it/s]

14:43:57.363 [ForkJoinPool-3-worker-3] WARN org.terrier.structures.indexing.Indexer - Adding an empty document to the index (2020.mir_conference-2020.1) - further warnings are suppressed


ir-lab-sose-2024/ir-acl-anthology-20240504-training documents: 100%|██████████| 126958/126958 [03:48<00:00, 555.96it/s] 


14:44:47.415 [ForkJoinPool-3-worker-3] WARN org.terrier.structures.indexing.Indexer - Indexed 2 empty documents


## Evaluation

In [5]:

pt.Experiment(
    retr_systems=[bm25, bm25_stem1,bm25_stem2, bm25_lem],
    topics=pt_dataset.get_topics('text'),
    qrels=pt_dataset.get_qrels(),
    names=['BM25', 'BM25_Stemmer1','BM25_Stemmer2', 'BM25_Lemmatizer'],
    eval_metrics=['recall_1000', 'ndcg_cut_5', 'ndcg_cut.10', 'recip_rank']
)



,name,recall_1000,ndcg_cut_5,ndcg_cut.10,recip_rank
0,BM25,0.825376,0.39365,0.374041,0.579877
1,BM25_Stemmer1,0.000000,0.00000,0.000000,0.000000
2,BM25_Stemmer2,0.000000,0.00000,0.000000,0.000000
3,BM25_Lemmatizer,0.000000,0.00000,0.000000,0.000000
